In [1]:
from elftools.elf.elffile import ELFFile
import frida
import sys
from time import sleep

In [2]:
def generate_function_list(binary):
    """
    Generate a list of all the functions defined by the target executable.
    """
    functions = []
    with open(binary, "rb") as f:
        elf = ELFFile(f)
        for section in elf.iter_sections():
            if section.header.sh_type == 'SHT_SYMTAB':
                for symbol in section.iter_symbols():
                    if symbol.entry.st_info.type == 'STT_FUNC':
                        functions.append(symbol.name)
    return functions

In [3]:
binary_path = "./test/test"
args = ["arg1", "arg2"]

function_list = generate_function_list(binary_path)
print(function_list)

['deregister_tm_clones', 'register_tm_clones', '__do_global_dtors_aux', 'frame_dummy', '__libc_start_main@GLIBC_2.34', 'puts@GLIBC_2.2.5', 'h', '_fini', 'f', '_start', 'g', 'main', '__cxa_finalize@GLIBC_2.2.5', '_init']


In [12]:
entries = []

def on_message(message, data):
    print(message)
    if message["type"] == "send" and message["payload"] != "done":
        function_name = message["payload"]["function"]
        function_args = message["payload"]["args"]
        entries.append((function_name, function_args))

# Run the binary
process = frida.spawn(binary_path, argv=[binary_path] + args)

sleep(1)

session = frida.attach(process)
for f in ['f','g','h']:
    script = session.create_script("""
        console.log(DebugSymbol.getFunctionByName('"""+f+"""'))
        Interceptor.attach(DebugSymbol.getFunctionByName('"""+f+"""'), {
            onEnter(args) {
                send({function: '"""+f+"""', args: args})
            },
            onLeave(retval) {
                send({function: '"""+f+"""', ret: retval})

            }
        });
    """)
    
    script.on("message", on_message)
    script.load()

frida.resume(process)

0x555c94926189
0x555c9492616a
0x555c94926149
{'type': 'send', 'payload': {'function': 'f', 'args': {}}}
{'type': 'send', 'payload': {'function': 'h', 'args': {}}}
{'type': 'send', 'payload': {'function': 'h', 'ret': '0x6'}}
Hello


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/frida/core.py", line 555, in _on_message
    callback(message, data)
  File "/tmp/ipykernel_28540/131413217.py", line 7, in on_message
    function_args = message["payload"]["args"]
KeyError: 'args'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/frida/core.py", line 555, in _on_message
    callback(message, data)
  File "/tmp/ipykernel_28540/131413217.py", line 7, in on_message
    function_args = message["payload"]["args"]
KeyError: 'args'


{'type': 'send', 'payload': {'function': 'f', 'ret': '0x6'}}
